In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import math

import multiprocessing
from multiprocessing.pool import ThreadPool

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelBinarizer

import keras
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

import cv2

import os


from sklearn.svm import SVC
from sklearn.preprocessing import LabelBinarizer
from sklearn import datasets, svm, metrics
from sklearn import decomposition

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import AdaBoostClassifier

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors.nearest_centroid import NearestCentroid

from sklearn import preprocessing

from cv2 import ml


In [ ]:
## Set local data path
dataPath = 'all'
localPath = ''
print(os.listdir(dataPath))

images = np.load(dataPath + '/train_images.npy', encoding="bytes")
labels = pd.read_csv(dataPath + '/train_labels.csv')
images.shape


In [ ]:
pd.Series(labels.groupby(['Category']).size()).sort_values(ascending=True)

In [ ]:
def preProcessImageObsolete(image, cutoff=127, maxContours=5):
    image = np.uint8(image)
    im = np.uint8(image)
    red, thresh = cv2.threshold(im, cutoff, 255, 0)
    im2, contours, hierarchy= cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.uint8(np.ones(im.shape))
    largest_contours = sorted(contours, key=cv2.contourArea)

    for ind, contour in enumerate(largest_contours[maxContours:]):
        mask = cv2.drawContours(mask, [largest_contours[ind]], -1, 0, -1)
        
    filteredImage = cv2.bitwise_and(thresh, thresh, mask=mask)
    ## plt.imshow(filteredImage)
    return filteredImage
    

In [ ]:
def preProcessImage(image, cutoff=127, areaCutoff=14, maxContours=4, fliplr=False):
    image = np.uint8(image)
    im = np.uint8(image)
    red, thresh = cv2.threshold(im, cutoff, 255, 0)
    im2, contours, hierarchy= cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros(im.shape, np.uint8)
    largest_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    for ind, contour in enumerate(largest_contours[:maxContours]):
        if cv2.contourArea(contour) > areaCutoff:
            x, y, w, h = cv2.boundingRect(contour)
            mask[y:y+h, x:x+w] = 255
        
    filteredImage = cv2.bitwise_and(image, image, mask=mask)
    #plt.imshow(filteredImage)
    #plt.figure()
    #plt.imshow(thresh)
    #plt.figure()
    #plt.imshow(mask)
    if fliplr:
        return np.fliplr(filteredImage).reshape((image.shape))
    return filteredImage.reshape((image.shape))

In [ ]:
#ONE SINGLE PICT

In [ ]:
num = 100
image = images[num][1].reshape(100, 100)
betterImage = preProcessImage(np.fliplr(image), maxContours=1)
plt.imshow(betterImage, cmap=plt.cm.binary)
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
print(labels.iloc[num])

In [ ]:
allData = pd.DataFrame(np.array(list(images[:,1]))).assign(label=labels['Category'])
num_classes = len(labels['Category'].unique())
lb = LabelBinarizer()
lb.fit(labels['Category'].unique())
valueCounts = labels['Category'].value_counts()

xTrain, xValid = train_test_split(allData, stratify=labels['Category'], test_size=0.05)
trainInds = xTrain.index
validInds = xValid.index
xTrainRaw = xTrain.drop('label', axis=1).values.reshape((xTrain.shape[0], 100, 100, 1))
xValidRaw = xValid.drop('label', axis=1).values.reshape((xValid.shape[0], 100, 100, 1))

yTrainString = labels.iloc[trainInds, 1].values
yValidString = labels.iloc[validInds, 1].values

yTrain = lb.transform(yTrainString)
yValid = lb.transform(yValidString)

In [ ]:
yValid

In [ ]:
epochs = 500
batch_size = 128

In [ ]:
pool = ThreadPool(multiprocessing.cpu_count())
xTrainUnflipped = pool.map(lambda im: preProcessImage(im).flatten(), [xTrainRaw[i] for i in range(xTrainRaw.shape[0])])
xTrainUnflipped = np.array(xTrainUnflipped)
xTrainFlipped = pool.map(lambda im: preProcessImage(im, fliplr=True).flatten(), [xTrainRaw[i] for i in range(xTrainRaw.shape[0])])
xTrainFlipped = np.array(xTrainFlipped)

xValidUnflipped = pool.map(lambda im: preProcessImage(im).flatten(), [xValidRaw[i] for i in range(xValidRaw.shape[0])])
xValidUnflipped = np.array(xValidUnflipped)
xValidFlipped = pool.map(lambda im: preProcessImage(im, fliplr=True).flatten(), [xValidRaw[i] for i in range(xValidRaw.shape[0])])
xValidFlipped = np.array(xValidFlipped)

xTrain = np.concatenate((xTrainUnflipped, xTrainFlipped), axis=0)
xValid = np.concatenate((xValidUnflipped, xValidFlipped), axis=0)

yTrain = np.concatenate((yTrain, yTrain), axis=0)
yValid = np.concatenate((yValid, yValid), axis=0)


pool.close()
pool.join()

In [ ]:
xTrain.shape

In [ ]:
#ONE EX TRAIN
num = 1011
plt.imshow(xTrain[num].reshape(100,100))
plt.figure()
plt.imshow(xTrainRaw[num].reshape(100,100))
yTrainString[num]

In [ ]:
trainIndsPerClass = {}
validIndsPerClass = {}

for i,label in enumerate(lb.classes_):
    trainIndsPerClass.update({label:np.where(yTrain[:,i] == 1)[0]})
    validIndsPerClass.update({label:np.where(yValid[:,i] == 1)[0]})

In [ ]:
for num in trainIndsPerClass['apple']:
    imagebefore = images[trainInds][num][1].reshape(100,100)
    plt.imshow(imagebefore, cmap=plt.cm.binary)
    plt.show()
    plt.close()
    image = xTrain[num].reshape(100, 100) 
    plt.figure()
    plt.imshow(image, cmap=plt.cm.binary)
    plt.show()
    plt.close()
    print(labels.iloc[trainInds[num],1])

In [ ]:
#to 1d 

In [ ]:
def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    includes smoothing
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    crossentropy = -np.sum(targets*np.log(predictions+1e-9))/N
    return crossentropy

In [ ]:
cross_entropy(np.array([1,2,3]), np.array([1,2,5]))

In [ ]:
def sigmoid(x):
    """
    Input x. Returns : sigmoid(x)
    """
    return 1 / (1 + math.exp(-x))
def dsigmoid(x):
    """
    Input x. Returns : derivative sigmoid(x) = sigmoid * (1 - sigmoid(x))
    """
    return 1 - sigmoid(1-x) 

In [ ]:
class myNN:
    def __init__(self, inN, outM, hiddenH, learningRate):
        self.inN = inN
        self.inNNum = len(inN)
        self.outM = outM #trues!
        self.outMNum = len(outM)
        self.hiddenH = hiddenH #this is numpy array indicating size of hidden layers
        self.hiddenNum = len(hiddenH)
        self.learningRate = learningRate 
        self.createRandom()
        self.nodes = []
        self.nodes.append(inN)
        for i in range(self.hiddenNum):
            self.nodes.append(np.ones(hiddenH[i])) #init ones
        self.nodes.append(np.ones(len(outM))) #init ones
    
    def createRandom(self):
        if self.hiddenNum == 0:
            self.weights = []
            self.weights.append(np.random.rand(self.inNNum+1,self.outMNum))
            
        else:
            self.weights = []
            self.weights.append(np.random.rand(self.inNNum+1,self.hiddenH[0]))
            for i in range(self.hiddenNum-1):
                self.weights.append(np.random.rand(self.hiddenH[i]+1, self.hiddenH[i+1]))
            self.weights.append(np.random.rand(self.hiddenH[-1]+1, self.outMNum))
        
    def run(self):
        vf = np.vectorize(sigmoid)
        for i in range(self.hiddenNum+1):
            self.nodes[i+1] = vf(np.append(self.nodes[i].T,1) @ self.weights[i])
            
    def errorvector(self):
        vf = np.vectorize(sigmoid)
        return vf(self.nodes[-1]) - self.outM
            
    def error(self):
        return cross_entropy(self.nodes[-1] , self.outM)
    
    def train(self):
        self.Ds = [] # reversed!
        vdf = np.vectorize(dsigmoid)
        vf = np.vectorize(sigmoid)
        self.Ds.append(np.diag(vdf(self.nodes[-1])))
        for i in range(self.hiddenNum):
            self.Ds.append(np.diag(np.append(vdf(self.nodes[-2-i]), 1)))
        """
        print("Ds:")
        print(self.Ds)
        """
        self.deltas = [] # reversed!
        self.deltas.append(self.Ds[0] @ self.errorvector())
        for i in range(self.hiddenNum):
            """
            print(self.Ds[i+1])
            print(self.weights[-1-i])
            print(self.deltas[-1])
            """
            self.deltas.append(self.Ds[i+1][:-1] @ vf(self.weights[-1-i]) @ self.deltas[-1])
        """  
        print("--")
        print(self.deltas)
        print(self.nodes)
        """
        for i in range(self.hiddenNum+1):
            """
            print(self.weights[i])
            print(self.deltas[-1-i])
            print(np.append(vf(self.nodes[-1-i]),1))
            print("-")
            print(self.deltas[-1-i].shape)
            print(np.append(vf(self.nodes[i]),1).shape)
            """
            self.weights[i] -= (self.learningRate * np.outer(self.deltas[-1-i], np.append(vf(self.nodes[i]),1))).T
    
    def trainEpochs(self, n):
        for _ in range(n):
            self.train()
            self.run()
            print(self.error())
    
    def show(self):
        print( "input layer: " +str(np.append(self.nodes[0],1)) )
        for i in range(self.hiddenNum):
            print(self.weights[i])
            print( str(i+1) + " layer: " + str(np.append(self.nodes[i+1],1)))
        print(self.weights[-1])
        print( "output layer: " + str(self.nodes[-1]))
            

In [ ]:
nett = myNN(inN=np.array([0.5,1.2,3.14,12.14,5.6]), outM=np.array([1,0,1,1,0]), hiddenH=np.array([2,3,2,5,10]), learningRate=0.01)
nett.run()
nett.show()

In [ ]:
nett.train()

In [ ]:
simpleNetwork = myNN(inN=np.array([0.5,1.2,3.14]), outM=np.array([1,0,1]), hiddenH=np.array([2,3,2]), learningRate=0.1)
simpleNetwork.run()

In [ ]:
simpleNetwork.show()

In [ ]:
simpleNetwork.error()

In [ ]:
simpleNetwork.train()

In [ ]:
simpleNetwork.run()
simpleNetwork.show()

In [ ]:
simpleNetwork.error()

In [ ]:
simpleNetwork.trainEpochs(20)

In [ ]:
withoutHidden = myNN(inN=np.array([0.5,1.2,3.14]), outM=np.array([1,0,1]), hiddenH=np.array([]), learningRate=0.1)
withoutHidden.run()

In [ ]:
withoutHidden.show()

In [ ]:
xTrain.shape

In [ ]:
#model Kaggle

In [ ]:
kaggle = myNN(inN = xTrain, outM = yTrain, hiddenH=[100, 100, 100, 100], learningRate=0.1)
kaggle.run()
kaggle.error()

In [ ]:
kaggle.train()
kaggle.run()
kaggle.error()